# <center> Image Upscaling using Sparse Denoising Autoencoders & GANs </center>
## <center> Part-1 Autoencoder </center>
### <center>Mohd Ayaan Anwar (2K19/CO/232) and Nakul Saroha (2K19/CO/238)</center>

# 0. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AI_Project

## 0.1. Fetch and Extract Data

In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [ ]:
!tar -xzvf lfw.tgz

In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw-names.txt

## 0.2. Import Libraries

In [ ]:
!pip install fastai --upgrade

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict
# visualization
from PIL import Image
import plotly.express as px


import numpy as np
import cv2
import glob

import tensorflow as tf
import keras
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image

import torch
from fastai.vision.all import *
from PIL import ImageFile

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import pickle

from tqdm import tqdm

# 1. EDA

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
path = 'lfw/'
data = ImageDataLoaders.from_folder(path, train='train',
                                   item_tfms=Resize(256),valid_pct=0.2,
                                   bs=64,seed=0)
data.show_batch()

In [ ]:
data.show_batch()

In [ ]:
data.show_batch()

In [ ]:
data.show_batch()

In [ ]:
lfw_allnames = pd.read_csv("lfw-names.txt", sep='\t', header=None, names=['name', 'images'])
lfw_allnames.head(), lfw_allnames.describe()

In [ ]:
print("Summary:")
print("There are " + 
      str(lfw_allnames.shape[0]) + 
      " unique celebrities in the entire dataset, of whom " + 
      str(sum(lfw_allnames.images > 1)) + 
      " are represented by multiple images. The entire number of images available is " + 
      str(sum(lfw_allnames.images)) + 
      ". The most represented celebrity is " + 
      str(lfw_allnames.iloc[lfw_allnames['images'].idxmax()][0]) + 
      ", with " + 
      str(max(lfw_allnames.images)) + 
      " unique images in the dataset.")

In [ ]:
image_paths = lfw_allnames.loc[lfw_allnames.index.repeat(lfw_allnames['images'])]
image_paths['image_path'] = 1 + image_paths.groupby('name').cumcount()
image_paths['image_path'] = image_paths.image_path.apply(lambda x: '{0:0>4}'.format(x))
image_paths['image_path'] = image_paths.name + "/" + image_paths.name + "_" + image_paths.image_path + ".jpg"
image_paths = image_paths.drop("images",1)

In [ ]:
# verify resolution of all images is consistent
widths = []
heights = []
files = image_paths.image_path
for file in files:
    path = "lfw/" + str(file)
    im = Image.open(path)
    widths.append(im.width)
    heights.append(im.height)

pd.DataFrame({'height':heights,'width':widths}).describe()

# all 250 x 250 resolution

We can observe that each image is of the resolution 250x250.

In [ ]:
image_paths['name'].value_counts()[:10].plot(kind = "bar")

We can see that there are some very well-represented figures among the top 10: generally political and generally male. This has important implications for the usefulness of this dataset in generalizations, which we consider in the conclusion. To draw this further into distinction, how many individuals are represented by a single image, compared to George W Bush's 530?

In [ ]:
ind_counts = image_paths.groupby('name').count().image_path
print(str(sum(ind_counts[ind_counts==1])) + " individuals, which is " +
      str(round(100*(sum(ind_counts[ind_counts==1])/sum(ind_counts)))) +
      "% of the total individuals considered, are only represented by a single image in this dataset.")

As a sanity check, and to check the directories are all correctly connected, we visualize a sample image:

In [ ]:
im = Image.open("lfw/" + str(image_paths.image_path[0]))
plt.imshow(im)

In [ ]:
df = lfw_allnames.sort_values(by='images', ascending=False)
fig = px.bar(df, x="name", y="images", color="images", title="Person and # Images")
fig.show()

# 2. Autoencoder

In [ ]:
face_images = glob.glob('lfw/**/*.jpg') #gives path

print(face_images[:2], len(face_images))

In [ ]:
with open('face_images_path.pickle','wb') as f:
  pickle.dump(face_images,f)

## 2.1. Load Images

In [ ]:
with open('face_images_path.pickle','rb') as f:
  face_images = pickle.load(f)

## 2.2. Randomly Sample 1200 images (1000 test + 200 validation)

In [ ]:
import random
random.seed(42)
face_images = random.sample(face_images, 1200)

In [ ]:
from multiprocessing import Pool
progress = tqdm(total= len(face_images), position=0)
def read(path):
  img = image.load_img(path, target_size=(256,256,3))
  img = image.img_to_array(img)
  img = img/255.
  progress.update(1)
  return img

p = Pool(10)
img_array = p.map(read, face_images)

In [ ]:
with open('img_array.pickle','wb') as f:
  pickle.dump(img_array, f)

In [ ]:
len(img_array)

## 2.3. Load Train, Test Data

In [ ]:
with open('img_array.pickle','rb') as f:
  img_array = pickle.load(f)

In [ ]:
len(img_array)

In [ ]:
plt.imshow(img_array[100])

In [ ]:
all_images = np.array(img_array)

In [ ]:
all_images.shape

In [ ]:
train_x = all_images[:1000]
val_x = all_images[1000:]

In [ ]:
len(train_x), len(val_x)

In [ ]:
#now we will make input images by lowering resolution without changing the size
def pixalate_image(image, scale_percent = 25):
  width = int(image.shape[1] * scale_percent / 100)
  height = int(image.shape[0] * scale_percent / 100)
  dim = (width, height)

  small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  # scale back to original size
  width = int(small_image.shape[1] * 100 / scale_percent)
  height = int(small_image.shape[0] * 100 / scale_percent)
  dim = (width, height)

  low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

  return low_res_image

In [ ]:
f, ax = plt.subplots(1, 2)
ax[0].set_title('Original Image')
ax[0].imshow(img_array[750])

ax[1].set_title('Pixelated Image')
ax[1].imshow(pixalate_image(img_array[750]))

In [ ]:
import matplotlib as mpl

def display_image_in_actual_size(img, type):

    dpi = mpl.rcParams['figure.dpi']
    im_data = img
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = 3*width / float(dpi), 3*height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')
    plt.title(type)
    plt.show()

display_image_in_actual_size(pixalate_image(img_array[500]), 'Pixelated')
display_image_in_actual_size(img_array[500], 'Original')

In [ ]:
train_x_px = []

for i in range(train_x.shape[0]):
  temp = pixalate_image(train_x[i,:,:,:])
  train_x_px.append(temp)

train_x_px = np.array(train_x_px)


# get low resolution images for the validation set
val_x_px = []

for i in range(val_x.shape[0]):
  temp = pixalate_image(val_x[i,:,:,:])
  val_x_px.append(temp)

val_x_px = np.array(val_x_px)

In [ ]:
train_x_px[100].shape, train_x[100].shape

# 2.4. Build Model

In [ ]:
Input_img = Input(shape=(256, 256, 3))  
    
#encoding architecture
x1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
x2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
x3 = MaxPool2D(padding='same')(x2)

x4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
x5 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
x6 = MaxPool2D(padding='same')(x5)

encoded = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)
#encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

# decoding architecture
x7 = UpSampling2D()(encoded)
x8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
x9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x8)
x10 = Add()([x5, x9])

x11 = UpSampling2D()(x10)
x12 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x11)
x13 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)
x14 = Add()([x2, x13])


decoded = Conv2D(3, (3, 3), padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x14)

autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
autoencoder.summary()
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=50, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint('AE_Models/autoencoder_checkpoint.h5',save_best_only=True)

## 2.5. Train Model

In [ ]:
history = autoencoder.fit(train_x_px,train_x,
            epochs=100,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper, model_checkpoint])

In [ ]:
autoencoder.save('autoencoder_batch.h5')
# autoencoder.save('autoencoder_all.h5')

# 3. Evaluate AE (trained on 1200 images)

In [ ]:
def pixelate_image(image, scale_percent = 25):
  width = int(image.shape[1] * scale_percent / 100)
  height = int(image.shape[0] * scale_percent / 100)
  dim = (width, height)

  small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  # scale back to original size
  width = int(small_image.shape[1] * 100 / scale_percent)
  height = int(small_image.shape[0] * 100 / scale_percent)
  dim = (width, height)

  low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

  return low_res_image

In [ ]:
autoencoder = keras.models.load_model('AE_Models/autoencoder_batch.h5')

In [ ]:
def read(path):
  img = image.load_img(path, target_size=(256,256,3))
  img = image.img_to_array(img)
  img = img/255.
  return img

In [ ]:
def evaluate(img):
    f, ax = plt.subplots(1, 2)
    ax[0].set_title('Input Image')
    ax[1].set_title('Output Image')

    test_img = np.array(pixelate_image(img))
    ax[0].imshow(test_img)
    test_img = np.expand_dims(test_img, axis=0)
    pred = autoencoder.predict(test_img)
    pred = np.squeeze(pred, axis=0)
    ax[1].imshow(pred)

## Images from the dataset

In [ ]:
evaluate(img_array[464])

In [ ]:
evaluate(img_array[600])

In [ ]:
evaluate(img_array[800])

## Images NOT in the dataset

In [ ]:
!ls Test_Images

In [ ]:
test1 = read('Test_Images/test1.jpg')
evaluate(test1)

In [ ]:
test2 = read('Test_Images/test2.jpg')
evaluate(test2)

In [ ]:
test3 = read('Test_Images/test3.jpg')
evaluate(test3)

# 4. Evaluate AE (trained on whole dataset)

In [ ]:
autoencoder2 = keras.models.load_model('AE_Models/autoencoder_all.h5')

In [ ]:
def read(path):
  img = image.load_img(path, target_size=(80,80,3))
  img = image.img_to_array(img)
  img = img/255.
  return img

In [ ]:
def evaluate(img):
    f, ax = plt.subplots(1, 2)
    ax[0].set_title('Input Image')
    ax[1].set_title('Output Image')
    img = cv2.resize(img, (80, 80))
    test_img = np.array(pixelate_image(img, scale_percent=40))
    ax[0].imshow(test_img)
    test_img = np.expand_dims(test_img, axis=0)
    pred = autoencoder2.predict(test_img)
    pred = np.squeeze(pred, axis=0)
    ax[1].imshow(pred)

## Images from the dataset

In [ ]:
evaluate(img_array[123])

In [ ]:
evaluate(img_array[297])

In [ ]:
evaluate(img_array[871])

## Images NOT in the dataset

In [ ]:
test1 = read('Test_Images/test1.jpg')
evaluate(test1)

In [ ]:
test2 = read('Test_Images/test2.jpg')
evaluate(test2)

In [ ]:
test3 = read('Test_Images/test3.jpg')
evaluate(test3)